# LCLS `cu_hxr` Modeling using vanilla Tao

In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'

%pylab --no-import-all inline

In [ ]:
from pytao import Tao
from lcls_live.datamaps.klystron import KlystronDataMap, existing_LCLS_klystrons_sector_station
from lcls_live.datamaps.tabular import TabularDataMap
from lcls_live.tools import isotime
from lcls_live import data_dir
import pandas as pd
import json
import os

In [ ]:
# Basic model with options
MODEL = 'cu_hxr'
OPTIONS = f'-slice OTR2:END -noplot'
INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'

# Make sure this exists
assert 'LCLS_LATTICE' in os.environ

INIT

## Tao

In [ ]:
tao = Tao(INIT)

## PVDATA

Load a dict of archived PV data. This is a simple dict of `pvname:value`

In [ ]:
PVDATA = json.load(open('data/PVDATA-2021-04-21T08:10:25.000000-07:00.json'))

# DataMaps 

## Quads

In [ ]:
def quad_pvinfo(tao, ele):
    """
    Returns dict of PV information for use in a DataMap
    """
    head = tao.ele_head(ele)
    attrs = tao.ele_gen_attribs(ele)
    device = head['alias']
    
    d = {}
    d['bmad_name'] = ele
    d['pvname_rbv'] = device+':BACT'
    d['pvname'] = device+':BDES'    
    d['bmad_factor'] = -1/attrs['L']/10
    d['bmad_attribute'] = 'b1_gradient'
    return d

quad_pvinfo(tao, 'QM01')

In [ ]:
quad_names = tao.lat_list('quad::*', 'ele.name', flags='-no_slaves')

dfq = pd.DataFrame([quad_pvinfo(tao, ele) for ele in quad_names])
dfq

In [ ]:
QUAD_DATAMAP = TabularDataMap(dfq, pvname='pvname', element='bmad_name', attribute = 'bmad_attribute', factor='bmad_factor')
#QUAD_DATAMAP.pvlist

In [ ]:
JSONFILE = os.path.join(data_dir, 'cu_hxr/quad_TabularDataMap.json')
QUAD_DATAMAP.to_json(JSONFILE)

## Klystrons

In [ ]:
klystron_names = tao.lat_list('overlay::K*', 'ele.name', flags='-no_slaves')

# Export to file and reload
klystron_datamaps = []
for i, identifier in enumerate(existing_LCLS_klystrons_sector_station):
    sector = identifier[0]
    station = identifier[1]
    
    JSONFILE = os.path.join(data_dir, f'cu/sector{sector}_station{station}_klystron_datamap.json')
    klystron_datamaps.append(KlystronDataMap.from_json(JSONFILE))


# Just take the ones that we have
klystron_datamaps = [k for k in klystron_datamaps if k.name in klystron_names]

## Linac and Bunch Compressors

In [ ]:
LINAC_JSON = os.path.join(data_dir, 'cu/linac_TabularDataMap.json')
LINAC_DATAMAP = TabularDataMap.from_json(LINAC_JSON)
LINAC_DATAMAP.data

## Energy measurements

In [ ]:
ENERGY_MEAS = os.path.join(data_dir, 'cu_hxr/tao_energy_measurements_TabularDataMap.json')
ENERGY_MEAS_DATAMAP = TabularDataMap.from_json(open(ENERGY_MEAS).read())
ENERGY_MEAS_DATAMAP.data

## Subboosters

In [ ]:
SUBBOSTER_DATAMAP = TabularDataMap.from_json(os.path.join(data_dir, 'cu/subboosters_TabularDataMap.json'))
SUBBOSTER_DATAMAP.data

## Beginning Twiss parameters

In [ ]:
BEGINNING_TWISS_DATAMAP = TabularDataMap.from_json(os.path.join(data_dir, 'cu/beginning_OTR2_TabularDataMap.json'))
BEGINNING_TWISS_DATAMAP.data

## Collect all DataMaps

In [ ]:
ALL_DATAMAPS = klystron_datamaps + [QUAD_DATAMAP, LINAC_DATAMAP, ENERGY_MEAS_DATAMAP, SUBBOSTER_DATAMAP, BEGINNING_TWISS_DATAMAP]

#ALL_DATAMAPS = [QUAD_DATAMAP]

def get_pvlist():
    pvlist = set()
    for dm in ALL_DATAMAPS:
        for pv in dm.pvlist:
            pvlist.add(pv) 
    return list(pvlist)

def get_bmad(pvdata):
    lines = []
    for dm in ALL_DATAMAPS:
        lines += dm.as_bmad(pvdata)
    return lines

def get_tao(pvdata):
    lines = []
    for dm in ALL_DATAMAPS:
        lines += dm.as_tao(pvdata)
    return lines

# Get a list of all PVs needed
ALL_PVS = get_pvlist()
len(ALL_PVS)

# Prepare all commands

In [ ]:
tao = Tao('-init $LCLS_LATTICE/bmad/models/cu_hxr/tao.init -slice OTR2:END -noplot')

In [ ]:
%%time
init_cmds = """
place floor energy
!set global plot_on = F
set global lattice_calc_on = F
set lattice model=design ! Reset the lattice
set ele quad::* field_master = T
""".split('\n')

cmds = get_tao(PVDATA)

final_cmds = """
set global lattice_calc_on = T
!set global plot_on = T
""".split('\n')

all_cmds = init_cmds + cmds + final_cmds

for cmd in all_cmds:
    tao.cmd(cmd)

In [ ]:
# Output collecting

outkeys = """ele.name
ele.ix_ele
ele.ix_branch
ele.a.beta
ele.a.alpha
ele.a.eta
ele.a.etap
ele.a.gamma
ele.a.phi
ele.b.beta
ele.b.alpha
ele.b.eta
ele.b.etap
ele.b.gamma
ele.b.phi
ele.x.eta
ele.x.etap
ele.y.eta
ele.y.etap
ele.s
ele.l
ele.e_tot
ele.p0c
ele.mat6
ele.vec0
""".split()
def get_output(tao):
    return {k:tao.lat_list('*', k) for k in outkeys}

In [ ]:
def evaluate_tao(tao, cmds):

    # Init
    for cmd in init_cmds:
        tao.cmd(cmd)

    for cmd in cmds:
        tao.cmd(cmd)
    
    # Turn lattice calc on
    for cmd in final_cmds:
        tao.cmd(cmd)    
        
    output = get_output(tao)
    
    return output

In [ ]:
O = evaluate_tao(tao, cmds)
O.keys()

In [ ]:
def plot_twiss(output, info=''):
    fig, ax = plt.subplots(figsize=(16,4))
    
    ax.plot(output['ele.s'], output['ele.a.beta'], label = r'$\beta_a$')
    ax.plot(output['ele.s'], output['ele.b.beta'], label = r'$\beta_b$')
    plt.legend()
    
    # Add energy to the rhs
    ax2 = ax.twinx()
    ax2.plot(output['ele.s'], output['ele.e_tot']/1e9, color='red')
    ax2.set_ylabel('Energy (GeV)')
    
    ax.set_xlabel('s (m)')
    ax.set_ylabel('Twiss Beta (m)')
    
    itime = isotime()
    efinal = output['ele.e_tot'][-1]
    plt.title(f'{info} Final energy: {efinal} GeV')
        
    return fig

plot_twiss(O);   

# Restore from the Archiver

In [ ]:
from lcls_live.archiver import lcls_archiver_restore
import os 
# Open an SSH tunnel in a terminal like:
# ssh -D 8080 cmayes@rhel6-64.slac.stanford.edu 
# And then set:
os.environ['http_proxy']='socks5h://localhost:8080'
os.environ['HTTPS_PROXY']='socks5h://localhost:8080'
os.environ['ALL_PROXY']='socks5h://localhost:8080'

In [ ]:
# Pick a proper time
ISOTIME = '2021-04-21T08:10:25.000000-07:00'
#ISOTIME = '2021-01-20T01:10:43.000000-07:00'
ARCHIVED_PVDATA = lcls_archiver_restore(ALL_PVS, ISOTIME)

# Save for restoring later
AOUT = f'data/PVDATA-{ISOTIME}.json'
json.dump(ARCHIVED_PVDATA, open(AOUT, 'w'))

# Some of the commands
cmds = get_tao(ARCHIVED_PVDATA)
cmds[0:10]

In [ ]:
fout = 'test.tao'
all_cmds = init_cmds + cmds + final_cmds
with open(fout, 'w') as f:
    for line in all_cmds:
        f.write(line+'\n')
os.path.abspath(fout)

In [ ]:
%%time
OUT = evaluate_tao(tao, cmds)

In [ ]:
plot_twiss(OUT)  

# Live loop

In [ ]:
import epics

def get_live():
    pvlist = get_pvlist()
    return dict(zip(pvlist, epics.caget_many(pvlist)))

In [ ]:
init_cmds = """
!place floor energy
!place top beta
!place middle eta
!place bottom layout 
!x-a all s
!sc
!set global plot_on = F
set global lattice_calc_on = F
set lattice model=design ! Reset the lattice
!set ele quad::* field_master = T
""".split('\n')

final_cmds = """
set global lattice_calc_on = T
!set global plot_on = T
!sc
""".split('\n')

In [ ]:
%%time
pvdata = get_live()
cmds = get_tao(pvdata)
output = evaluate_tao(tao, cmds)


In [ ]:
from IPython.display import clear_output

In [ ]:
def plot1():
    """
    Get live EPICS data and plot
    """
    pvdata = get_live()
    cmds = get_tao(pvdata)
    output = evaluate_tao(tao, cmds)    
    
    clear_output(wait=True)  
    fig = plot_twiss(output, info=isotime())  

    fig.axes[0].set_ylim(0,1000)
    

    plt.show()
    

In [ ]:
%%time
plot1()

In [ ]:
# Run forever
while True:
    plot1()